# Nebenläufigkeit (*englisch:* Concurrency)

## Nebenläufigkeit

> Die Nebenläufigkeit, mitunter auch Parallelität (englisch concurrency) genannt, ist in der Informatik die Eigenschaft eines Systems, mehrere Aufgaben, Berechnungen, Anweisungen oder Befehle gleichzeitig ausführen zu können. Es kann sich dabei um völlig unabhängige Anweisungen handeln, bis hin zur gemeinsamen Bearbeitung einer Aufgabe. Sie können dabei auch miteinander interagieren (z. B. um Zwischenergebnisse auszutauschen).
> -- <cite>[Wikipedia][1]</cite>

[1]: https://de.wikipedia.org/wiki/Nebenl%C3%A4ufigkeit

Es ist zu unterscheiden zwischen *scheinbarer* und *echter* Parallelität. Im ersten Fall werden zum Beispiel beim *Multitasking* einzelne Aufgaben von einem Prozessor seriell abgearbeitet, erscheinen aber durch Task-Wechsel parallel. Echte Parallelität hingegen findet auf Rechnern mit mehrern Prozessoren beziehungsweise Mehr-Kern-CPUs und in verteilten Rechner-Netzwerken statt.

## Parallelisierbarkeit

Programme können durch die Nutzung von mehreren Ausführungsfäden (Threads) erheblich beschleunigt werden. Jedoch ist hierbei zu beachten, dass es einige Fallstricke gibt, insbesondere dann, wenn einzelne Programmteile voneinander abhängen und auf Datenaustausch angewiesen sind. In diesem Fall ist es notwendig geeignete Synchronisationsverfahren anzuwenden.

## Motivation

Welche Probleme kann ich mit Nebenläufigkeit lösen?

Ganz offensichtlich lassen Programmabläufe, die sich gut parallelisieren lassen, wie z.B. mathematische Operationen bei der Bildbearbeitung erheblich beschleunigen. Darauf ausgerichtet gibt es mittlerweile spezielle Hardware (GPUs), die sich z.B. mit *OpenCL* oder *CUDA* nutzen lässt.

Ein weiteres Beispiel ist die asynchrone Ausführung von Netzwerkoperationen. Hierbei werden im Hintergrund Daten geladen, während dem Benutzer bereits eine grafische Oberfläche angezeigt wird. Dabei werden die Daten im Nachhinein eingefügt und verbessern dadurch das Benutzererlebnis, da nicht auf die Ankunft der Daten gewartet werden muss. Diese Technik ist auch als *latency hiding* bekannt.

## Arten von Nebenläufigkeit

### Prozesse und Threads

#### Prozess

> Ein Prozess (auch Task oder Programminstanz genannt) ist ein Computerprogramm zur Laufzeit. Genauer ist ein Prozess die konkrete Instanziierung eines Programms zu dessen Ausführung innerhalb eines Rechnersystems, ergänzt um weitere (Verwaltungs-)Informationen und Ressourcenzuteilungen des Betriebssystems für diese Ausführung.
> -- <cite>[Wikipedia][2]</cite>

#### Thread

> Ein (Kernel-)Thread ist ein sequentieller Abarbeitungslauf innerhalb eines Prozesses und teilt sich mit den anderen vorhandenen Threads des zugehörigen Prozesses eine Reihe von Betriebsmitteln.
> -- <cite>[Wikipedia][4]</cite>

![Multiprocessing and Multithreading](multiprocessing_threading.png "Multiprocessing and Multithreading")

### Multiprocessing

Multiprocessing ermöglicht die nebenläufige Ausführung von mehreren *Prozessen*, auch bekannt als Multitasking:

> Der Begriff Multitasking bzw. Mehrprozessbetrieb bezeichnet die Fähigkeit eines Betriebssystems, mehrere Aufgaben (Tasks) (quasi-)nebenläufig auszuführen. [..] Die verschiedenen Prozesse werden in so kurzen Abständen immer abwechselnd aktiviert, dass der Eindruck der Gleichzeitigkeit entsteht. [..] Besitzt ein Computer mehrere CPU-Kerne, so dass er mehrere Aufgaben echt-gleichzeitig ausführen kann, so spricht man von Multiprocessing.
> -- <cite>[Wikipedia][1]</cite>

### Multithreading

Multithreading bezeichnet die (quasi-)gleichzeitige Abarbeitung mehrerer Threads innerhalb *eines einzelnen Prozesses*.

> Im Gegensatz zum Multitasking, bei dem mehrere unabhängige Programme voneinander abgeschottet quasi-gleichzeitig ausgeführt werden, sind die Threads eines Anwendungsprogramms nicht voneinander abgeschottet und können somit durch sogenannte Race Conditions Fehler verursachen, die durch Synchronisation vermieden werden müssen.
> -- <cite>[Wikipedia][3]</cite>

[1]: https://de.wikipedia.org/wiki/Multitasking
[2]: https://de.wikipedia.org/wiki/Prozess_(Informatik)
[3]: https://de.wikipedia.org/wiki/Multithreading
[4]: https://de.wikipedia.org/wiki/Thread_(Informatik)

## Nebenläufigkeit in Python

### Das [Global Interpreter Lock (GIL)](https://docs.python.org/3/glossary.html#term-global-interpreter-lock)

In Python (genauer [CPython](https://docs.python.org/3/glossary.html#term-CPython)) gibt es einen Mechanismus namens *Global Interpreter Lock*, also in etwa *globale Interpreter Sperre*.

Dieser Mechanismus wurde eingeführt, um die Implementierung des Bytecode Interpreters zu vereinfachen, indem nur ein (Kernel-) Thread jeweils Bytecode ausführen darf, während der Rest warten muss. Dies garantiert, dass das interne Datenmodell (z.B. `dict`s) sicher sind vor nebenläufigem Zugriff.

Es gab in der Vergangenheit Versuche Interpreter mit granularerem Locking-Strategien, also nur einer Sperre der akut betroffenen Datenstrukturen. Diese waren jedoch nicht erfolgreich, da die Performance im weit verbreiteten Ein-Prozessor Use-Case erheblich litt. Mittlerweile geht man davon aus, dass es eine Implementierung, welche dieses Dilemma löst zu aufwendig und damit zu schwer zu warten wäre.

## Python APIs

### Multiprocessing

Das Multiprocessing Paket erlaubt es *Unterprozesse* zu erzeugen. (Englisch: *fork or spawn a subprocess*). Da es hierbei auf der Prozessebene fungiert wird dadurch effektiv das GIL umgangen. Das Paket ermöglich lokale und verteilte (auf anderen Rechnern, *remote*) Nebenläufigkeit und funktioniert sowohl auf Unix als auch auf Windows.

### Nutzung von Multiprocessing

Zuerst wird die Klasse `Process` importiert:

In [5]:
from multiprocessing import Process

Danach können wir einen Prozess generieren und starten:

In [4]:
def f(name):
    print('hello', name)
    
p = Process(target=f, args=('alice',))
p.start()
p.join()

hello alice


Ein erweitertes Beispiel, bei dem zusätzlich die beteiligten Prozess-IDs ausgegeben werden:

In [7]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 3269803
process id: 3270043
function f
module name: __main__
parent process: 3270043
process id: 3377676
hello bob


### Datenaustausch
#### Queues

Mit Hilfe von `Queues` können Daten zwischen Prozessen geteilt werden:

In [22]:
from multiprocessing import Process, Queue
from time import sleep

def f1(q):
    sleep(0.1)
    q.put(q.get() + [42])
    print('f1() done')
    
def f2(q):
    sleep(0.2)
    q.put(q.get() + [None])
    print('f2() done')
    
def f3(q):
    sleep(0.3)
    q.put(q.get() + ['hello'])
    print('f3() done')
    
if __name__ == '__main__':
    q = Queue(3)
    q.put([])
    p1 = Process(target=f1, args=(q,))
    p2 = Process(target=f2, args=(q,))
    p3 = Process(target=f3, args=(q,))
    p1.start()
    p2.start()
    p3.start()
    p1.join()
    p2.join()
    p3.join()
    print(q.get())

f1() done
f2() done
f3() done
[42, None, 'hello']


#### Pipes

Ein etwas aufwändigerer, aber dafür mächtigerer Mechanismus sind `Pipe`s. Damit können bidirektionale Verbindungen zwischen zwei Prozessen aufgebaut werden:

In [23]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


Der Aufruf von `Pipe()` liefert hierbei die beiden Enden der Verbindung zurück, welche man mit den Methoden `send()` und `recv()` nutzen kann. Versucht man ein Ende der Verbindung gleichzeitig mit verschiedenen Prozessen oder Threads zu nutzen kann es zu Datenkorruption kommen.

#### Synchronisation

Prozesse können vergleichbar zu Threads mit Hilfe eines Locks synchronisiert werden, zum Beispiel um sicherzustellen, dass nur ein Prozess auf der Standardausgabe schreibt:

In [24]:
from multiprocessing import Process, Lock

def f(l, i):
    l.acquire()
    try:
        print('hello world', i)
    finally:
        l.release()

if __name__ == '__main__':
    lock = Lock()

    for num in range(10):
        Process(target=f, args=(lock, num)).start()

hello world 0
hello world 1
hello world 2
hello world 3
hello world 4
hello world 5
hello world 6
hello world 7
hello world 8
hello world 9


Ohne Locks wird die Ausgabe wild durcheinander gewürfelt:

In [27]:
from multiprocessing import Process

def f(i):
    print('hello world', i)

if __name__ == '__main__':
    for num in range(10):
        Process(target=f, args=(num, )).start()

hello world hello worldhello world0hello world hello worldhello world  1
24hello world 
  
5
3hello world
6hello worldhello world

   879




### Dokumentation

Mehr Information zu `multiprocessing` findet sich in der offiziellen [Dokumentation](https://docs.python.org/3/library/multiprocessing.html).

### Threading

Das `threading` Paket implementiert User-Space Threads in Python. Allerdings ist aufgrund des *GIL* immer nur ein Thread im Interpreter lauffähig. Um die volle Leistungsfähigkeit von Multi-Core Maschinen ausschöpfen zu können empfiehlt sich der Einsatz des `multiprocessing` Pakets. Für Programme, welche IO-*Bound* sind (d.h. die Performance wird durch den Zugriff auf IO-Resourcen ausgebremst) ist der Einsatz von `threading` trotzdem empfehlenswert, da hier das *GIL* nicht zum tragen kommt.

Erstellen eines Threads mit einer Funktion:

In [29]:
import logging
import threading
import time

def f1(name):
    logging.info("Thread %s: starting", name)
    time.sleep(1)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=f1, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    x.join()
    logging.info("Main    : all done")

15:05:54: Main    : before creating thread
15:05:54: Main    : before running thread
15:05:54: Thread 1: starting
15:05:54: Main    : wait for the thread to finish
15:05:55: Thread 1: finishing
15:05:55: Main    : all done


Erstellen eines Threads durch Ableitung der `Thread` Klasse:

In [32]:
import logging
import time
from threading import Thread

class MyThread(Thread):
    def __init__(self, name, *args, **kwargs):
        self.__name = name
        super().__init__(*args, **kwargs)
        
    def run(self):
        logging.info("Thread %s: starting", self.__name)
        time.sleep(2)
        logging.info("Thread %s: finishing", self.__name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = MyThread(1)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    x.join()
    logging.info("Main    : all done")

15:14:00: Main    : before creating thread
15:14:00: Main    : before running thread
15:14:00: Thread 1: starting
15:14:00: Main    : wait for the thread to finish
15:14:02: Thread 1: finishing
15:14:02: Main    : all done


Mit Hilfe von `is_alive()` kann überprüft werden, ob ein Thread noch läuft:

In [33]:
x.is_alive()

False

Möchte man ein Thread automatisch bei Programmende beenden lassen, dann dann man das Argument `daemonic=True` and `Thread` übergeben.

Hierbei ist jedoch zu beachten, dass sogenannte *daemon threads* abrupt beendet werden, d.h. geöffnete Resourcen wie z.B. Dateien oder Datenbankverbindungen werden nicht geschlossen. Um das zu ermöglichen ist es notwendig den Thread auf eine andere Art und Weise zu stoppen. Dazu eignet sich z.B. die `Event` Klasse, mit der geprüft werden, ob der Thread beendet werden soll oder nicht:

In [35]:
import logging
import time
from threading import Thread, Event

class MyThread(Thread):
    def __init__(self, name, *args, **kwargs):
        self.__name = name
        self.__count = 0
        self.__stop = Event()
        super().__init__(*args, **kwargs)

    def stop(self):
        self.__stop.set()

    def run(self):
        logging.info("Thread %s: starting", self.__name)
        while True:
            if self.__stop.is_set():
                break
            logging.info("Thread %s: tick %s", self.__name, self.__count)
            self.__count = self.__count + 1
            time.sleep(1)
        logging.info("Thread %s: finishing", self.__name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = MyThread(1)
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : sleeping a bit")
    time.sleep(5)
    logging.info("Main    : stopping thread")
    x.stop()
    logging.info("Main    : waiting for thread to exit")
    x.join()
    logging.info("Main    : all done")

15:23:07: Main    : before creating thread
15:23:07: Main    : before running thread
15:23:07: Thread 1: starting
15:23:07: Thread 1: tick 0
15:23:07: Main    : sleeping a bit
15:23:08: Thread 1: tick 1
15:23:09: Thread 1: tick 2
15:23:10: Thread 1: tick 3
15:23:11: Thread 1: tick 4
15:23:12: Thread 1: tick 5
15:23:12: Main    : stopping thread
15:23:12: Main    : waiting for thread to exit
15:23:13: Thread 1: finishing
15:23:13: Main    : all done


Weitere Synchronisationsmethoden sind `Lock`, `RLock`, `Condition`, `Semaphore`, `Timer` und `Barrier`. `Lock` und `RLock` funktionieren analog zu `multiprocessing`. Die Besonderheit an `RLock` ist, dass es *reentrant* ist, d.h. mehrere Aufrufe der `acquire()` Methode führen nicht zu einem Deadlock, wie es z.B. bei `Lock` passieren könnte.

Grundsätzlich unterstützen alle Synchronisationsmethoden das [context management protocol](https://docs.python.org/3/library/threading.html#with-locks):

```python
with some_lock:
    # do something...
```

ist dabei gleichbedeutend mit:

```python
some_lock.acquire()
try:
    # do something...
finally:
    some_lock.release()
```

#### Conditions

Condition variables sind besonders interessant für *Producer-Consumer*-Anwendungen. D.h. Ein oder mehrere Threads erzeugen (produzieren) Daten, welche von einem oder mehreren anderen Threads gelesen (konsumiert) werden. Der produzierende Thread nutzt dafür die `notify()` und `notify_all()` Methoden, während die konsumierenden Threads mit `wait()` und `wait_for()` auf den gewünschten Zustand warten können.

Dies ist ein gängiges Pattern bei der Programmierung nebenläufiger Anwendungen. Hier soll darauf jedoch nicht weiter eingegangen werden. Weiterführende Literatur findet sich z.B. auf [Wikipedia](https://de.wikipedia.org/wiki/Erzeuger-Verbraucher-Problem).

#### Semaphores

Semaphoren sind eine der ältesten Synchronisationsprimitiven, erfunden von Edsger W. Dijkstra. Sie funktionieren so ähnlich wie Locks, implementieren jedoch zusätzlich einen Zähler.

#### Timer

Mit einem Timer ist es möglich eine verzögerte Aktion auszuführen. Der Thread kann vor dem eigentlichen Aufruf der Aktion beendet werden. Eine beispielhafte Anwendung wäre zum Beispiel ein "long press". Der Benutzer drückt eine Taste und hält diese gedrückt. Der Timer startet und führt z.B. nach 2 Sekunden eine Aktion aus. Löst der Benutzer vor Ablauf der 2 Sekunden die Taste wird der Timer gestoppt und die eigentliche Tastenaktion wird ausgeführt.

#### Barrier

Mit einer Barrier kann eine feste Anzahl von Threads an einem gemeinsamen Synchronisationspunkt gesammelt werden. Jeder Thread ruft am Ende seines Ablaufs die `wait()` Methode auf, welche blockierend ist, d.h. der Thread muss hier warten. Wenn alle Threads `wait()` aufgerufen haben, wird die Schranke gelöst und die Threads können weiter arbeiten. 

### Datenaustausch

Zum Datenaustausch gibt es ein eigenes [`queue` Paket](https://docs.python.org/3/library/queue.html#module-queue). Im Gegensatz zu der Queue aus der Standardbibliothek ist diese Variante Thread-safe und erzeugt keine Race-Conditions oder Datenkorruption.

In [37]:
import threading, queue

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()

# turn-on the worker thread
threading.Thread(target=worker, daemon=True).start()

# send ten task requests to the worker
for item in range(10):
    q.put(item)
print('All task requests sent\n', end='')

# block until all tasks are done
q.join()
print('All work completed')

All task requests sent
Working on 0
Finished 0
Working on 1
Finished 1
Working on 2
Finished 2
Working on 3
Finished 3
Working on 4
Finished 4
Working on 5
Finished 5
Working on 6
Finished 6
Working on 7
Finished 7
Working on 8
Finished 8
Working on 9
Finished 9
All work completed


### Dokumentation

Mehr Information zu `threading` findet sich in der offiziellen [Dokumentation](https://docs.python.org/3/library/threading.html).

## Übung: Primzahlen

Implementiere einen einfachen Algorithmus um bis zu einer fixen Zahl `N` alle Primzahlen zu finden. Als Beispiel bis 10 wären das 2, 3, 5 und 7. Verwendet werden kann z.B. die [Probedivision](https://de.wikipedia.org/wiki/Probedivision).

Miss die Performance (Ausführgeschwindigkeit) der Implementierung mit einer Messgröße `N`, so dass das Ergebnis bei ein paar Sekunden liegt.

Implementiere rein fakultativ den Algorithmus "[Sieb des Eratosthenes](https://de.wikipedia.org/wiki/Sieb_des_Eratosthenes)". Miss auch hier die Performance und vergleiche sie mit der naiven Implementierung.

Nutze nun das Paket `multiprocessing`, um die naive Implementierung zu beschleunigen. Miss wiederum die Performance für verschiedene Werte von `N`. Was für Schlüsse kann man hieraus ziehen? Ist die parallelisierte Variante generell schneller? Warum oder warum nicht?